In [ ]:
import os
import sys

# Thêm thư mục gốc của dự án vào sys.path
project_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
if project_root not in sys.path:
    sys.path.insert(0, project_root)

print("Project root added to sys.path:", project_root)

In [ ]:
import numpy as np
import tensorflow as tf
from src.model import build_transfer_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam

<span style="color: cyan;font-weight: bold;">Định nghĩa đường dẫn<span>

In [ ]:
PROCESSED_DIR = '../data/processed'
MODEL_PATH = '../models/transfer_learning_model.h5'

<span style="color: cyan;font-weight: bold;">Tải dữ liệu đã tiền xử lý<span>

In [ ]:
X_train = np.load(f"{PROCESSED_DIR}/X_train.npy")
y_train = np.load(f"{PROCESSED_DIR}/y_train.npy")
X_val = np.load(f"{PROCESSED_DIR}/X_val.npy")
y_val = np.load(f"{PROCESSED_DIR}/y_val.npy")

<span style="color: cyan;font-weight: bold;">Chuyển ảnh xám thành RGB<span>

In [ ]:
X_train_rgb = np.repeat(X_train, 3, axis=-1)
X_val_rgb = np.repeat(X_val, 3, axis=-1)

<span style="color: cyan;font-weight: bold;">Xây dựng và biên dịch mô hình<span>

In [ ]:
model = build_transfer_model()
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

<span style="color: cyan;font-weight: bold;">Huấn luyện ban đầu với base_model đóng băng<span>

In [ ]:
early_stop = EarlyStopping(monitor='val_loss', patience=10)
history = model.fit(datagen.flow(X_train_rgb, y_train, batch_size=32),
                    validation_data=(X_val_rgb, y_val),
                    epochs=50,
                    callbacks=[early_stop])

<span style="color: cyan;font-weight: bold;">Data Augmentation<span>

In [ ]:
datagen = ImageDataGenerator(rotation_range=10, width_shift_range=0.1, height_shift_range=0.1, horizontal_flip=True)
datagen.fit(X_train_rgb)

<span style="color: cyan;font-weight: bold;">Huấn luyện mô hình<span>

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
lr_reduce = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3)

history = model.fit(
    datagen.flow(X_train_rgb, y_train, batch_size=32),
    validation_data=(X_val_rgb, y_val),
    epochs=30,
    callbacks=[early_stop, lr_reduce],
    class_weight=class_weight_dict
)


<span style="color: cyan;font-weight: bold;">Lưu mô hình<span>

In [ ]:
model.save(MODEL_PATH)

<span style="color: cyan;font-weight: bold;">tạo file training_history.png<span>

In [ ]:
import os
import matplotlib.pyplot as plt

# Đường dẫn đến thư mục reports/figures
FIGURES_DIR = "reports/figures"
os.makedirs(FIGURES_DIR, exist_ok=True)  # Tạo thư mục nếu chưa có

# Vẽ và lưu biểu đồ
plt.plot(history.history['accuracy'], label='train_accuracy')
plt.plot(history.history['val_accuracy'], label='val_accuracy')
plt.legend()
plt.savefig(f"{FIGURES_DIR}/training_history.png")
plt.show()
